<a href="https://colab.research.google.com/github/Fmarcelneves/Twitter_Network_Analytics/blob/main/Machine_learning_model_construction_Felipe_M_Neves_political_theme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Portuguese Tweets for Sentiment Analysis using nltk and sklearn</h1>

In [ ]:
!pip install xgboost

<h2>Load libraries</h2>

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
# from nltk.tokenize import sent_tokenize (Tokenization)
from nltk.probability import FreqDist
from nltk.metrics import ConfusionMatrix
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
import  xgboost as xgb

In [ ]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<h2>Load training dataset</h2>

<h4>Load and prepare X_train, y_train</h4>

In [ ]:
NeutralHash = pd.read_csv('TweetsNeutralHash.csv',encoding='utf-8')
NeutralNews = pd.read_csv('TweetsNeutralNews.csv',encoding='utf-8')
Political_Neg_Pos = pd.read_csv('TweetsWithTheme.csv',encoding='utf-8')
Classes = pd.read_csv('Train3Classes.csv', encoding='utf-8', sep=';')

In [ ]:
NeutralHash  = NeutralHash[['tweet_text', 'sentiment']]
NeutralHash.columns = ['texto', 'classificacao']

In [ ]:
NeutralHash.shape

(15727, 2)

In [ ]:
NeutralHash = NeutralHash.drop_duplicates().reset_index(drop=True)

In [ ]:
NeutralHash.shape

(15515, 2)

In [ ]:
NeutralNews  = NeutralNews[['tweet_text', 'sentiment']]
NeutralNews.columns = ['texto', 'classificacao']

In [ ]:
NeutralNews.shape

(37556, 2)

In [ ]:
NeutralNews = NeutralNews.drop_duplicates().reset_index(drop=True)

In [ ]:
NeutralNews.shape

(37509, 2)

In [ ]:
Political_Neg_Pos  = Political_Neg_Pos[['tweet_text', 'sentiment']]
Political_Neg_Pos.columns = ['texto', 'classificacao']

In [ ]:
Political_Neg_Pos.shape

(61591, 2)

In [ ]:
Political_Neg_Pos = Political_Neg_Pos.drop_duplicates().reset_index(drop=True)

In [ ]:
Political_Neg_Pos.shape

(61283, 2)

In [ ]:
NeutralHash = NeutralHash.sample(8000) #14000 #8000

In [ ]:
NeutralHash.shape

(8000, 2)

In [ ]:
NeutralNews = NeutralNews.sample(20000) #14000 #20000

In [ ]:
NeutralNews.shape

(20000, 2)

In [ ]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].shape

(32566, 2)

In [ ]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].shape

(28717, 2)

In [ ]:
Political_Pos = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].sample(20000) #18000 #20000 #14000

In [ ]:
Political_Pos.shape

(20000, 2)

In [ ]:
Classes_Pos  = Classes[['tweet_text', 'sentiment']]
Classes_Pos.columns = ['texto', 'classificacao']

In [ ]:
Classes_Pos['classificacao'] = Classes_Pos['classificacao'].map({0: 'Negativo', 1: 'Positivo', 2: 'Neutro'})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Classes_Pos = Classes_Pos.loc[Classes_Pos['classificacao'] == 'Positivo'].sample(8000) #10000 #14000 #8000

In [ ]:
Classes_Pos.shape

(8000, 2)

In [ ]:
Political_Neg = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].sample(28000)

In [ ]:
Political_Neg.shape

(28000, 2)

In [ ]:
dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, Classes_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
dataset

,texto,classificacao
0,"#FATO EU MUITO MENDIGUEI, MAS CREIO QUE HOJE A...",Neutro
1,"Não tá fácil pra ninguém, não quer dizer que n...",Neutro
2,Vocês também fazem twittes mentais quando estã...,Neutro
3,"#BBdebate Ninguém está notando isso, mas desde...",Neutro
4,"Jogando em Rio Pardo, a AMF de Marau venceu o ...",Neutro
...,...,...
83667,@AleYoussef @Haddad_Fernando voto útil no Ciro...,Negativo
83668,Novamente!! :( :( https://t.co/pF6sjoJIre,Negativo
83669,"não quero que o bozo vença, nunca estive tão p...",Negativo
83670,Se o petista #LulaHadad vencer a corrida eleit...,Negativo


In [ ]:
dataset.shape

(83672, 2)

In [ ]:
#dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

In [ ]:
#dataset = pd.concat([NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

<h4>Exploratory Analysis</h4>

In [ ]:
dataset.sample(5)

,texto,classificacao
21205,"Com incerteza eleitoral, risco-país do Brasil ...",Neutro
73961,@hackradt Setores da sociedade brasileira com ...,Negativo
62978,"O que vai ser de Roni sem a sua ""ciganita""? :(...",Negativo
16725,"Dólar opera em queda, negociado abaixo dos R$ ...",Neutro
35536,"""Nós nunca subvertemos dados, candidato"" (Bonn...",Positivo


In [ ]:
series = dataset['classificacao'].value_counts()
series

Negativo    28000
Neutro      27993
Positivo    27679
Name: classificacao, dtype: int64

In [ ]:
%%capture
!pip install plotly --upgrade
from plotly import __version__
print (__version__)
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly as py
#import plotly.figure_factory as ff
from plotly import tools
#import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:
pd.options.plotting.backend = "plotly"


In [ ]:
series.plot(kind='bar', title='Número de tweets classificados para cada sentimento considerado', 
labels={"value": "Frequências", "index": "Sentimentos"}, color = ['Negativo', 'Neutro', 'Positivo'], 
color_discrete_sequence =['#EF553B', '#00CC96', '#636EFA'])

<h2>Text Preprocessing</h2>

<h4>Planning</h4>
<ul>
  <li>Noise Removal</li>
    <ul>
      <li>Remove urls from text (http(s), www)</li>
      <li>Remove a regex pattern (hashtags, notations)</li>
      <li>Prepare text, replace emoticons ":)) :) :D :(" to positive_emoticon or negative_emoticon or neutral_emoticon tags</li>
    </ul>
  <li>Tokenize text with nltk</li>
  <li>Object Standardization</li>
  <li>Remove stopwords (noise removal and dimensionality reduction)</li>
  <li>Lexicon Normalization</li>
    <ul>
      <li>Lemmatization (not implemented...)</li>
      <li>Stemming (dimensionality reduction)</li>
    </ul>
</ul> 

<h4>Examples of emoticons</h4>

<h4>Noise Removal</h4>

Remove urls from text (http(s), www)

In [ ]:
def _remove_url(data):
    ls = []
    words = ''
    regexp1 = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    regexp2 = re.compile('www?.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    
    for line in data:
        urls = regexp1.findall(line)

        for u in urls:
            line = line.replace(u, ' ')

        urls = regexp2.findall(line)

        for u in urls:
            line = line.replace(u, ' ')
            
        ls.append(line)
    return ls

In [ ]:
dataset['texto'] = _remove_url(dataset['texto'])

Remove a regex pattern (hashtags, notations)

In [ ]:
def _remove_regex(data, regex_pattern):
    ls = []
    words = ''
    
    for line in data:
        matches = re.finditer(regex_pattern, line)
        
        for m in matches: 
            line = re.sub(m.group().strip(), '', line)

        ls.append(line)

    return ls

In [ ]:
# hashtags
regex_pattern = '#[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
# notations
regex_pattern = '@[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
# rt
regex_pattern = 'RT[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
 emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [ ]:
# emoji pattern 
regex_pattern = emoji_pattern 
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

 EU MUITO MENDIGUEI, MAS CREIO QUE HOJE APRENDI A LIÇÃO EM NOME DO SENHOR JESUS, POIS QUEM FUGIU DE VOCÊ SEM DAR EXPLICAÇÃO ALGUMA, JAMAIS FOI DIGNA DE…  
Não tá fácil pra ninguém, não quer dizer que não está divertido. 
Vocês também fazem twittes mentais quando estão sem celular? 
 Ninguém está notando isso, mas desde que o Rever voltou ao time a produção caiu! Enquanto ele esteva fora por lesão o sistema defensivo do Flamengo esteve muito mais sólido. 
Jogando em Rio Pardo, a AMF de Marau venceu o Nadas Branco, de virada, por 05 a 04. Semana que vem o jogo será contra o Lagoa Futsal, em Marau. O jogo será domingo, dia 16, as 18h.         


Prepare text, replace emoticons ":)) :) :D :(" to positive_emoticon or negative_emoticon or neutral_emoticon

In [ ]:
def _replace_emoticons(data, emoticon_list):
    ls = []

    for line in data:
        for exp in emoticon_list:
            line = line.replace(exp, emoticon_list[exp])

        ls.append(line)

    return ls

In [ ]:
emoticon_list = {':))': 'emoticon_positivo', ':)': 'emoticon_positivo', ':D': 'emoticon_positivo', ':(': 'emoticon_negativo', ':((': 'emoticon_negativo', '8)': 'emoticon_neutro'}

In [ ]:
dataset['texto']= _replace_emoticons(dataset['texto'], emoticon_list)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

 EU MUITO MENDIGUEI, MAS CREIO QUE HOJE APRENDI A LIÇÃO EM NOME DO SENHOR JESUS, POIS QUEM FUGIU DE VOCÊ SEM DAR EXPLICAÇÃO ALGUMA, JAMAIS FOI DIGNA DE…  
Não tá fácil pra ninguém, não quer dizer que não está divertido. 
Vocês também fazem twittes mentais quando estão sem celular? 
 Ninguém está notando isso, mas desde que o Rever voltou ao time a produção caiu! Enquanto ele esteva fora por lesão o sistema defensivo do Flamengo esteve muito mais sólido. 
Jogando em Rio Pardo, a AMF de Marau venceu o Nadas Branco, de virada, por 05 a 04. Semana que vem o jogo será contra o Lagoa Futsal, em Marau. O jogo será domingo, dia 16, as 18h.         


<h5 style="color:red;">Note: We can remove or replace with tags some noisy words or special characters like ✨💖🎈🎉🎊❤️, but we need to evaluate if they are important to express some sentiment</h5>

In [ ]:
dataset

,texto,classificacao
0,"EU MUITO MENDIGUEI, MAS CREIO QUE HOJE APREND...",Neutro
1,"Não tá fácil pra ninguém, não quer dizer que n...",Neutro
2,Vocês também fazem twittes mentais quando estã...,Neutro
3,"Ninguém está notando isso, mas desde que o Re...",Neutro
4,"Jogando em Rio Pardo, a AMF de Marau venceu o ...",Neutro
...,...,...
83667,voto útil no Ciro está muito viável por cont...,Negativo
83668,Novamente!! emoticon_negativo emoticon_negativo,Negativo
83669,"não quero que o bozo vença, nunca estive tão p...",Negativo
83670,Se o petista vencer a corrida eleitoral terem...,Negativo


In [ ]:
nan_value = float("NaN")
dataset.replace("", nan_value, inplace=True)

<h4>Tokenize text with nltk</h4>

In [ ]:
def _tokenize_text(data):
    ls = []

    for line in data:
        tokens = wordpunct_tokenize(line)
        ls.append(tokens)

    return ls

In [ ]:
dataset_texto_tokens = _tokenize_text(dataset['texto'])

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['EU', 'MUITO', 'MENDIGUEI', ',', 'MAS', 'CREIO', 'QUE', 'HOJE', 'APRENDI', 'A', 'LIÇÃO', 'EM', 'NOME', 'DO', 'SENHOR', 'JESUS', ',', 'POIS', 'QUEM', 'FUGIU', 'DE', 'VOCÊ', 'SEM', 'DAR', 'EXPLICAÇÃO', 'ALGUMA', ',', 'JAMAIS', 'FOI', 'DIGNA', 'DE', '…']
['Não', 'tá', 'fácil', 'pra', 'ninguém', ',', 'não', 'quer', 'dizer', 'que', 'não', 'está', 'divertido', '.']
['Vocês', 'também', 'fazem', 'twittes', 'mentais', 'quando', 'estão', 'sem', 'celular', '?']
['Ninguém', 'está', 'notando', 'isso', ',', 'mas', 'desde', 'que', 'o', 'Rever', 'voltou', 'ao', 'time', 'a', 'produção', 'caiu', '!', 'Enquanto', 'ele', 'esteva', 'fora', 'por', 'lesão', 'o', 'sistema', 'defensivo', 'do', 'Flamengo', 'esteve', 'muito', 'mais', 'sólido', '.']
['Jogando', 'em', 'Rio', 'Pardo', ',', 'a', 'AMF', 'de', 'Marau', 'venceu', 'o', 'Nadas', 'Branco', ',', 'de', 'virada', ',', 'por', '05', 'a', '04', '.', 'Semana', 'que', 'vem', 'o', 'jogo', 'será', 'contra', 'o', 'Lagoa', 'Futsal', ',', 'em', 'Marau', '.', 'O', 'jo

In [ ]:
#dataset_texto_tokens = list(filter(None, dataset_texto_tokens))

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['EU', 'MUITO', 'MENDIGUEI', ',', 'MAS', 'CREIO', 'QUE', 'HOJE', 'APRENDI', 'A', 'LIÇÃO', 'EM', 'NOME', 'DO', 'SENHOR', 'JESUS', ',', 'POIS', 'QUEM', 'FUGIU', 'DE', 'VOCÊ', 'SEM', 'DAR', 'EXPLICAÇÃO', 'ALGUMA', ',', 'JAMAIS', 'FOI', 'DIGNA', 'DE', '…']
['Não', 'tá', 'fácil', 'pra', 'ninguém', ',', 'não', 'quer', 'dizer', 'que', 'não', 'está', 'divertido', '.']
['Vocês', 'também', 'fazem', 'twittes', 'mentais', 'quando', 'estão', 'sem', 'celular', '?']
['Ninguém', 'está', 'notando', 'isso', ',', 'mas', 'desde', 'que', 'o', 'Rever', 'voltou', 'ao', 'time', 'a', 'produção', 'caiu', '!', 'Enquanto', 'ele', 'esteva', 'fora', 'por', 'lesão', 'o', 'sistema', 'defensivo', 'do', 'Flamengo', 'esteve', 'muito', 'mais', 'sólido', '.']
['Jogando', 'em', 'Rio', 'Pardo', ',', 'a', 'AMF', 'de', 'Marau', 'venceu', 'o', 'Nadas', 'Branco', ',', 'de', 'virada', ',', 'por', '05', 'a', '04', '.', 'Semana', 'que', 'vem', 'o', 'jogo', 'será', 'contra', 'o', 'Lagoa', 'Futsal', ',', 'em', 'Marau', '.', 'O', 'jo

<h4>Object Standardization</h4>

In [ ]:
def _apply_standardization(tokens, std_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() in std_list:
                word = std_list[word.lower()]
                
            new_tokens.append(word) 
            
        ls.append(new_tokens)

    return ls

In [ ]:
# create your own list
std_list = {'eh': 'é', 'vc': 'você', 'vcs': 'vocês','tb': 'também', 'tbm': 'também', 'obg': 'obrigado', 'gnt': 'gente', 'q': 'que', 'n': 'não', 'cmg': 'comigo', 'p': 'para', 'ta': 'está', 'to': 'estou', 'vdd': 'verdade'}

In [ ]:
dataset_texto_tokens = _apply_standardization(dataset_texto_tokens, std_list)

In [ ]:
# check data
print(dataset_texto_tokens[4], dataset_texto_tokens[1008])

['Jogando', 'em', 'Rio', 'Pardo', ',', 'a', 'AMF', 'de', 'Marau', 'venceu', 'o', 'Nadas', 'Branco', ',', 'de', 'virada', ',', 'por', '05', 'a', '04', '.', 'Semana', 'que', 'vem', 'o', 'jogo', 'será', 'contra', 'o', 'Lagoa', 'Futsal', ',', 'em', 'Marau', '.', 'O', 'jogo', 'será', 'domingo', ',', 'dia', '16', ',', 'as', '18h', '.'] ['Supervisor', 'de', 'Vendas', 'Salário', ':', 'A', 'combinar', '+', 'Benefícios', 'Local', ':', 'Osasco', 'Mais', 'informações', 'sobre', 'a', 'vaga', ':', 'Quer', 'mais', 'vagas', '?', 'Acesse', ':']


<h4>Remove stopwords (noise removal and dimensionality reduction)</h4>

Remove stopwords

In [ ]:
def _remove_stopwords(tokens, stopword_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() not in stopword_list:
                new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [ ]:
stopword_list = []

In [ ]:
# get nltk portuguese stopwords
nltk_stopwords = nltk.corpus.stopwords.words('portuguese')

In [ ]:
# You can also add stopwords manually instead of loading from the database. Generally, we add stopwords that belong to this context.
stopword_list.append('é')
stopword_list.append('vou')
stopword_list.append('que')
stopword_list.append('tão')
stopword_list.append('...')
stopword_list.append('«')
stopword_list.append('➔')
stopword_list.append('|')
stopword_list.append('//')
stopword_list.append('/')
stopword_list.append('||') 
stopword_list.append(':')
stopword_list.append('-')
stopword_list.append(';')
stopword_list.append('»')
stopword_list.append('…')
stopword_list.append(')')
stopword_list.append('(')
stopword_list.append('uai') 
stopword_list.append(']')
stopword_list.append('[')
stopword_list.append('((')
stopword_list.append('))')
stopword_list.append('emoticon_positivo') 
stopword_list.append('emoticon_negativo')
stopword_list.append('emoticon_neutro')
# 'expression from the mineiros (MG/Brazil)'

<h5 style="color:red;">Note: Numbers, some punctuations (.,;:) and special characters such as (_- = &%) are not relevant to express some sentiment, they can be considered noisy words. You can treat them here.</h5>
<h5 style="color:red;">noisy_words = ['.', '?', '!', ':', ',', ';', '(', ')', '-']</h5>
<h5 style="color:red;">Note: ? and ! can be important to express sentiment.</h5>

In [ ]:
# join all stopwords
stopword_list.extend(nltk_stopwords)
#stopword_list.extend(custom_stopwords)

In [ ]:
# remove duplicate stopwords (unique list)
stopword_list = list(set(stopword_list))

In [ ]:
dataset_texto_tokens = _remove_stopwords(dataset_texto_tokens, stopword_list)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['MENDIGUEI', ',', 'CREIO', 'HOJE', 'APRENDI', 'LIÇÃO', 'NOME', 'SENHOR', 'JESUS', ',', 'POIS', 'FUGIU', 'DAR', 'EXPLICAÇÃO', 'ALGUMA', ',', 'JAMAIS', 'DIGNA']
['tá', 'fácil', 'pra', 'ninguém', ',', 'quer', 'dizer', 'divertido', '.']
['fazem', 'twittes', 'mentais', 'celular', '?']
['Ninguém', 'notando', ',', 'desde', 'Rever', 'voltou', 'time', 'produção', 'caiu', '!', 'Enquanto', 'esteva', 'lesão', 'sistema', 'defensivo', 'Flamengo', 'sólido', '.']
['Jogando', 'Rio', 'Pardo', ',', 'AMF', 'Marau', 'venceu', 'Nadas', 'Branco', ',', 'virada', ',', '05', '04', '.', 'Semana', 'vem', 'jogo', 'contra', 'Lagoa', 'Futsal', ',', 'Marau', '.', 'jogo', 'domingo', ',', 'dia', '16', ',', '18h', '.']


<h4>Lexicon Normalization</h4>

Lemmatization (not implemented...)

Stemming (dimensionality reduction)

In [ ]:
def _apply_stemmer(tokens):
    ls = []
    stemmer = nltk.stem.RSLPStemmer()

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            word = str(stemmer.stem(word))
            new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [ ]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
dataset_texto_stemm = _apply_stemmer(dataset_texto_tokens)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['MENDIGUEI', ',', 'CREIO', 'HOJE', 'APRENDI', 'LIÇÃO', 'NOME', 'SENHOR', 'JESUS', ',', 'POIS', 'FUGIU', 'DAR', 'EXPLICAÇÃO', 'ALGUMA', ',', 'JAMAIS', 'DIGNA']
['tá', 'fácil', 'pra', 'ninguém', ',', 'quer', 'dizer', 'divertido', '.']
['fazem', 'twittes', 'mentais', 'celular', '?']
['Ninguém', 'notando', ',', 'desde', 'Rever', 'voltou', 'time', 'produção', 'caiu', '!', 'Enquanto', 'esteva', 'lesão', 'sistema', 'defensivo', 'Flamengo', 'sólido', '.']
['Jogando', 'Rio', 'Pardo', ',', 'AMF', 'Marau', 'venceu', 'Nadas', 'Branco', ',', 'virada', ',', '05', '04', '.', 'Semana', 'vem', 'jogo', 'contra', 'Lagoa', 'Futsal', ',', 'Marau', '.', 'jogo', 'domingo', ',', 'dia', '16', ',', '18h', '.']


<h4>Dataset preparation</h4>
Untokenize text (transform tokenized text into string list)

In [ ]:
def _untokenize_text(tokens):
    ls = []

    for tk_line in tokens:
        new_line = ''
        
        for word in tk_line:
            new_line += word + ' '
            
        ls.append(new_line)
        
    return ls

In [ ]:
dataset_texto_tokens = _untokenize_text(dataset_texto_tokens)

In [ ]:
dataset_texto_stemm = _untokenize_text(dataset_texto_stemm)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_stemm[i])

mendig , crei hoj aprend liç nom senh jesu , poi fug dar explic algum , jamal dign 
tá fácil pra ninguém , qu diz divert . 
faz twitt ment celul ? 
ninguém not , desd rev volt tim produç caiu ! enquant estev les sistem defens flameng sól . 
jog rio pard , amf marau venc nad branc , vir , 05 04 . seman vem jog contr lago futs , marau . jog doming , dia 16 , 18h . 


<h2>Implementing a Model</h2>


In [ ]:
tweets = np.array(dataset_texto_stemm,  dtype=object)
tweets

array(['mendig , crei hoj aprend liç nom senh jesu , poi fug dar explic algum , jamal dign ',
       'tá fácil pra ninguém , qu diz divert . ',
       'faz twitt ment celul ? ', ...,
       'quer boz venç , nunc preocup brasil tô agor , tô preocup comig outr pesso ! real tô med dess ele ',
       'petist venc corr eleitor brasil presid presidi pi espéci , moral agum govern quadrilh petist prov ser exim roub destru escrupul ',
       'faz letr ant letr pt ingl ador curs quer lecion '], dtype=object)

In [ ]:
classes = dataset["classificacao"].values
classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Negativo', 'Negativo',
       'Negativo'], dtype=object)

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neural_network import MLPClassifier

### Random Forest

In [ ]:
model_RF = Pipeline([
    ('vect', CountVectorizer(max_features=200)),
    ('tfidf', TfidfTransformer()),
    ('pca', TruncatedSVD(n_components=150)),
    ('clf', RandomForestClassifier(random_state=42, n_jobs=-1))
])

In [ ]:
model_xgb = Pipeline([
    ('vect', CountVectorizer()),
    #('tfidf', TfidfTransformer()),
    #('pca', TruncatedSVD(n_components=200)),
    ('clf', xgb.XGBClassifier(n_estimators=1000, n_jobs=-1, nthread=-1))
])

### Random Forest Model with cross-validation

In [ ]:
#predictions_RF = cross_val_predict(model_RF, tweets, classes, cv = 10)

In [ ]:
#print(classification_report(classes, predictions_RF))

In [ ]:
#for doc, predictions_RF, y in zip(tweets[0:10], predictions_RF[0:10], classes[0:10]):
#    print('%r => %s [%s]' % (doc, predictions_RF, y))

### Random forest model with Hold-out

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, classes, test_size=0.33, random_state=42)

In [ ]:
model_xgb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing

In [ ]:
predictions_xgb = model_xgb.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_xgb))

              precision    recall  f1-score   support

    Negativo       0.75      0.69      0.72      9262
      Neutro       0.74      0.88      0.81      9122
    Positivo       0.66      0.59      0.63      9228

    accuracy                           0.72     27612
   macro avg       0.72      0.72      0.72     27612
weighted avg       0.72      0.72      0.72     27612



In [ ]:
model_RF.fit(X_train, y_train)
predictions_RF = model_RF.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_RF))

              precision    recall  f1-score   support

    Negativo       0.65      0.68      0.67      9262
      Neutro       0.73      0.81      0.76      9122
    Positivo       0.58      0.48      0.52      9228

    accuracy                           0.66     27612
   macro avg       0.65      0.66      0.65     27612
weighted avg       0.65      0.66      0.65     27612



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_RF[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'quer trabalh nov ele ' => Negativo [Negativo]
'lei auxíli ' => Neutro [Neutro]
'tô trist amanhã famíl vai cheg urn faz 17 ' => Negativo [Negativo]
'quer fic marin hoj ' => Negativo [Negativo]
'ach vil pouc ameaç apresent companiom legal jodi whittak cativ , divert empolg , tud pra ser ótim protagon . be welcom , new doc ! ' => Negativo [Positivo]
"areth franklin ' grav doent ', diz amig famíl can " => Neutro [Neutro]
'vaa presid !!! brilh ' => Neutro [Positivo]
'assist program eleitor igual . ce maravilh , paul alckmin . ai paul vam lá cearens vem cá vem tud m paul ' => Neutro [Positivo]
'foc constru ' => Neutro [Neutro]
'datafalh golp haddad & lt 3 ¹³ 4 % 9 agor 13 % . vox popul cut haddad apoi lul 22 %. send reconhec . vai cresc pesquis golp ! ' => Negativo [Positivo]
'cap preserv campanh , bolsonar silenc vic . vej ' => Neutro [Neutro]
'mei trist , poi ach am chef vot bozonir hoj ô mulh , pra quê ? democrac ' => Negativo [Negativo]
'vej muit gent sab baix víde daqu twitt , ent deix

### Neural Network - Multilayer Perceptron

<img src="https://st2.ning.com/topology/rest/1.0/file/get/1140047167?profile=original" width="500"/>

In [ ]:
model_NN = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('pca', TruncatedSVD(n_components=200)), # 100000 linhas por 150 colunas (features)
    ('clf', MLPClassifier(hidden_layer_sizes=(100,50), #2 camadas escondidas sendo a primeira com 100 neurônios e a segunda com 50 neurônios 
                          random_state=42, 
                          activation='relu', #Teste outras funções de ativação. Ex. tanh, entre outras (relu).
                          max_iter=1000,
                          early_stopping=True))
])

In [ ]:
#predictions_NN = cross_val_predict(model_NN, tweets, classes)

In [ ]:
#print(classification_report(classes, predictions_NN))

In [ ]:
#for doc, prediction, y in zip(tweets[0:30], resultados[0:30], classes[0:30]):
#    print('%r => %s [%s]' % (doc, prediction, y))

In [ ]:
#param_grid = { 
#    'vect__ngram_range': [(1, 1), (1, 2)],
#    'tfidf__use_idf': (True, False),
#    'clf__n_estimators': [100, 200, 300, 400, 500],
#    'clf__max_features': ['auto', 'sqrt', 'log2'],
#    'clf__max_depth' : [None, 4,5,6,7,8],
#    'clf__criterion' :['gini', 'entropy']
#}

### Neural network model with Hold-out

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, classes, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

(56060,)

In [ ]:
X_test.shape

(27612,)

In [ ]:
model_NN.fit(X_train, y_train)
predictions_NN = model_NN.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_NN))

              precision    recall  f1-score   support

    Negativo       0.68      0.77      0.72      9262
      Neutro       0.86      0.81      0.83      9122
    Positivo       0.63      0.59      0.61      9228

    accuracy                           0.72     27612
   macro avg       0.73      0.72      0.72     27612
weighted avg       0.73      0.72      0.72     27612



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_NN[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'quer trabalh nov ele ' => Negativo [Negativo]
'lei auxíli ' => Neutro [Neutro]
'tô trist amanhã famíl vai cheg urn faz 17 ' => Negativo [Negativo]
'quer fic marin hoj ' => Negativo [Negativo]
'ach vil pouc ameaç apresent companiom legal jodi whittak cativ , divert empolg , tud pra ser ótim protagon . be welcom , new doc ! ' => Positivo [Positivo]
"areth franklin ' grav doent ', diz amig famíl can " => Neutro [Neutro]
'vaa presid !!! brilh ' => Positivo [Positivo]
'assist program eleitor igual . ce maravilh , paul alckmin . ai paul vam lá cearens vem cá vem tud m paul ' => Positivo [Positivo]
'foc constru ' => Positivo [Neutro]
'datafalh golp haddad & lt 3 ¹³ 4 % 9 agor 13 % . vox popul cut haddad apoi lul 22 %. send reconhec . vai cresc pesquis golp ! ' => Positivo [Positivo]
'cap preserv campanh , bolsonar silenc vic . vej ' => Neutro [Neutro]
'mei trist , poi ach am chef vot bozonir hoj ô mulh , pra quê ? democrac ' => Negativo [Negativo]
'vej muit gent sab baix víde daqu twitt , en

### Implementing a Linear Classifier (Logistic Regression) with Hold-Out

In [ ]:
#parameters = {
#    'vect__ngram_range': [(1, 1), (1, 2)],
#    'tfidf__use_idf': (True, False),
#    'clf__solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),
#    'clf__penalty': ('l1', 'l2', 'elasticnet', 'none'),
#}

In [ ]:
#gs_model_LR = GridSearchCV(model_LR, parameters, cv = 5, iid = False, n_jobs = -1)
#gs_model_LR.fit(X_train, y_train)

In [ ]:
#print(gs_model_LR.best_params_)

In [ ]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, classes, test_size=0.35, random_state=42)

In [ ]:
model_LR.fit(X_train, y_train)
predictions_LR = model_LR.predict(X_test)

In [ ]:
X_train.shape, X_test.shape

((54386,), (29286,))

In [ ]:
print(classification_report(y_test, predictions_LR))

              precision    recall  f1-score   support

    Negativo       0.73      0.77      0.75      9807
      Neutro       0.86      0.86      0.86      9666
    Positivo       0.70      0.65      0.67      9813

    accuracy                           0.76     29286
   macro avg       0.76      0.76      0.76     29286
weighted avg       0.76      0.76      0.76     29286



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_LR[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'quer trabalh nov ele ' => Negativo [Negativo]
'lei auxíli ' => Neutro [Neutro]
'tô trist amanhã famíl vai cheg urn faz 17 ' => Negativo [Negativo]
'quer fic marin hoj ' => Negativo [Negativo]
'ach vil pouc ameaç apresent companiom legal jodi whittak cativ , divert empolg , tud pra ser ótim protagon . be welcom , new doc ! ' => Positivo [Positivo]
"areth franklin ' grav doent ', diz amig famíl can " => Neutro [Neutro]
'vaa presid !!! brilh ' => Neutro [Positivo]
'assist program eleitor igual . ce maravilh , paul alckmin . ai paul vam lá cearens vem cá vem tud m paul ' => Positivo [Positivo]
'foc constru ' => Neutro [Neutro]
'datafalh golp haddad & lt 3 ¹³ 4 % 9 agor 13 % . vox popul cut haddad apoi lul 22 %. send reconhec . vai cresc pesquis golp ! ' => Positivo [Positivo]
'cap preserv campanh , bolsonar silenc vic . vej ' => Neutro [Neutro]
'mei trist , poi ach am chef vot bozonir hoj ô mulh , pra quê ? democrac ' => Negativo [Negativo]
'vej muit gent sab baix víde daqu twitt , ent de

### SVM model with Hold out

In [ ]:
model_SVC = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

In [ ]:
model_SVC = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(random_state=42)),
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, classes, test_size=0.35, random_state=42)

In [ ]:
X_train.shape

(54386,)

In [ ]:
X_test.shape

(29286,)

In [ ]:
#model_SVC.fit(X_train, y_train)
#predictions_SVC = model_SVC.predict(predictions_SVC)

In [ ]:
#print(classification_report(y_test, predictions_LR))

In [ ]:
#for doc, prediction, y in zip(X_test[0:20], predictions_SVC[0:20], y_test[0:20]):
#    print('%r => %s [%s]' % (doc, prediction, y))

### Importing models

In [ ]:
import pickle

In [ ]:
#filename = 'RF_sa_model.sav'
pickle.dump(model_RF, open('RF_model_we', 'wb'))

In [ ]:
#filename = 'RF_sa_model.sav'
pickle.dump(model_NN, open('NN_model_we', 'wb'))

In [ ]:
#filename = 'LR_sa_model.sav'
pickle.dump(model_LR, open('LR_model_we', 'wb'))

In [ ]:
loaded_model = pickle.load(open('RF_model_we', 'rb'))

In [ ]:
loaded_model = pickle.load(open('NN_model_we', 'rb'))

In [ ]:
loaded_model = pickle.load(open('LR_model_we', 'rb'))

In [ ]:
model_LR.predict(X_test)

array(['Negativo', 'Neutro', 'Negativo', ..., 'Negativo', 'Negativo',
       'Positivo'], dtype=object)

In [ ]:
predictions = loaded_model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

    Negativo       0.73      0.77      0.75      9807
      Neutro       0.86      0.86      0.86      9666
    Positivo       0.70      0.65      0.67      9813

    accuracy                           0.76     29286
   macro avg       0.76      0.76      0.76     29286
weighted avg       0.76      0.76      0.76     29286

